In [1]:
import json
import csv
import os
import glob
import pandas as pd

In [2]:
with open("../visData.json") as f:
    data = json.load(f)

In [3]:
file_names =  []
for item in data:
    file_names.append('..' + item['Highlight Data'])

In [4]:
combined_csv = pd.concat([pd.read_csv(f) for f in file_names ])

combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [5]:
json_data = pd.read_json('visData.json')
#renames and dropped unwanted columns
json_data = json_data.rename(columns = {'article_sha256': 'Article ID'}).drop(columns = ['Highlight Data','Plain Text','Visualization Link','Article Link', 'ID','articleHash'])
json_data

,Article ID,Title,Author,Date
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
1,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,"Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
2,be0b18a87d4370fa579180ef26dcb7080598f27f9ec761...,SARS-CoV-2 Can Live on Plastic and Steel for 2...,Kerry Grens,2020-03-12 18:56:55+00:00
3,47990959103662e94e796d979018922afddc880fb4b867...,2005 CIA Report on Coronavirus Pandemic Discov...,Lyubov Stepushova,2020-03-17 18:56:55+00:00
4,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...,US military may have brought coronavirus to Wu...,Straits Times,2020-03-17 18:56:55+00:00


In [6]:
combined = pd.read_csv('combined_csv.csv')

In [7]:
combined.head(3)

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN


In [8]:
combined['Start, End'] = combined[['Start','End']].apply(tuple,axis = 1)

In [9]:
#merge both csv and json dataframes together
full_data = pd.merge(left = combined, right = json_data, on = 'Article ID')
#whats Target Text?
full_data.head()

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text,"Start, End",Title,Author,Date
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
3,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H1,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
4,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H2,Holistic,Vague Sourcing,-2.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00


In [10]:
full_data

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text,"Start, End",Title,Author,Date
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
3,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H1,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
4,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H2,Holistic,Vague Sourcing,-2.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...,R3,Reasoning,False Dilemma,-2.500000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US military may have brought coronavirus to Wu...,Straits Times,2020-03-17 18:56:55+00:00
114,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...,R4,Reasoning,Equivocation,-2.500000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US military may have brought coronavirus to Wu...,Straits Times,2020-03-17 18:56:55+00:00
115,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...,R5,Reasoning,Appeal to Ignorance,-3.531968,[],-1.0,-1.0,It might be US army who brought the epidemic t...,"(-1.0, -1.0)",US military may have brought coronavirus to Wu...,Straits Times,2020-03-17 18:56:55+00:00
116,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...,R6,Reasoning,Other logical fallacies,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US military may have brought coronavirus to Wu...,Straits Times,2020-03-17 18:56:55+00:00


In [11]:
#Changes Date from DateType object to string type
full_data.Date = full_data.Date.astype(str)
data = full_data.groupby(['Article ID', 'Credibility Indicator Category','Credibility Indicator Name','Credibility Indicator ID','Start, End']).agg(lambda x:list(x)[0])

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 118 entries, ('3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4ec66b45681c86c9301', 'Evidence', 'Open to evidence', 'E0', (-1.0, -1.0)) to ('be0b18a87d4370fa579180ef26dcb7080598f27f9ec76181f2cfd851f320da06', 'Probability', 'Open to evidence', 'P1', (2274.0, 2496.0))
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Points                       118 non-null    float64
 1   Indices of Label in Article  118 non-null    object 
 2   Start                        118 non-null    float64
 3   End                          118 non-null    float64
 4   target_text                  36 non-null     object 
 5   Title                        118 non-null    object 
 6   Author                       118 non-null    object 
 7   Date                         118 non-null    object 
dtypes: float64(3), object(5)
memory usage: 9.3+ KB


In [13]:
data['Date'] = pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 118 entries, ('3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4ec66b45681c86c9301', 'Evidence', 'Open to evidence', 'E0', (-1.0, -1.0)) to ('be0b18a87d4370fa579180ef26dcb7080598f27f9ec76181f2cfd851f320da06', 'Probability', 'Open to evidence', 'P1', (2274.0, 2496.0))
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   Points                       118 non-null    float64            
 1   Indices of Label in Article  118 non-null    object             
 2   Start                        118 non-null    float64            
 3   End                          118 non-null    float64            
 4   target_text                  36 non-null     object             
 5   Title                        118 non-null    object             
 6   Author                       118 non-null    object             
 7   Date              

In [14]:
data

Points  \
Article ID                                         Credibility Indicator Category Credibility Indicator Name Credibility Indicator ID Start, End                   
3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4e... Evidence                       Open to evidence           E0                       (-1.0, -1.0)     -1.000000   
                                                   Holistic                       Qualified Source           H1                       (-1.0, -1.0)      2.000000   
                                                                                                             H2                       (-1.0, -1.0)      1.000000   
                                                                                                             H3                       (-1.0, -1.0)      2.000000   
                                                                                                             H4                       (-1.0, -1.0)      2.000000   
...                                                                                                                                                          ...   
be0b18a87d4370fa579180ef26dcb7080598f27f9ec7618... Holistic                       Qualified Source           H2                       (-1.0, -1.0)      2.000000   
                                                                                                             H3                       (-1.0, -1.0)      1.000000   
                                                                                  Vague Sourcing             H4                       (-1.0, -1.0)     -2.000000   
                                                   Probability                    Acknowledges uncertainty   P0                       (2204.0, 2389.0)  0.757298   
                                                                                  Open to evidence           P1                       (2274.0, 2496.0)  0.333333   

                                                                                                                                                                              Indices of Label in Article  \
Article ID                                         Credibility Indicator Category Credibility Indicator Name Credibility Indicator ID Start, End                                                            
3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4e... Evidence                       Open to evidence           E0                       (-1.0, -1.0)                                                     []   
                                                   Holistic                       Qualified Source           H1                       (-1.0, -1.0)                                                     []   
                                                                                                             H2                       (-1.0, -1.0)                                                     []   
                                                                                                             H3                       (-1.0, -1.0)                                                     []   
                                                                                                             H4                       (-1.0, -1.0)                                                     []   
...                                                                                                                                                                                                   ...   
be0b18a87d4370fa579180ef26dcb7080598f27f9ec7618... Holistic                       Qualified Source           H2                       (-1.0, -1.0)                                                     []   
                                                                                                             H3                       (-1.0, -1.0)                              

## Group by "Column Name" and Return a Data Frame

We fill the NaN values with 0 and round the summed values to the nearest integer

In [15]:
full_data.head()

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text,"Start, End",Title,Author,Date
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
3,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H1,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
4,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H2,Holistic,Vague Sourcing,-2.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00


In [25]:
full_data[full_data['Article ID'] == '7d1747829a65cd4dad0faf30a704875a125e9decce3b3bcc82f151193923aad8']

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text,"Start, End",Title,Author,Date
21,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,E0,Evidence,Alternative explanation,-1.456280,[],-1.0,-1.0,NaN,"(-1.0, -1.0)","Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
22,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,E1,Evidence,Alternative explanation,-0.847997,[],-1.0,-1.0,NaN,"(-1.0, -1.0)","Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
23,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,E2,Evidence,Statistical uncertainty,-0.678072,"[914, 915, 916, 917, 918, 919, 920, 921, 922, ...",914.0,1060.0,NaN,"(914.0, 1060.0)","Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
24,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,E3,Evidence,Statistical uncertainty,-0.500000,"[333, 334, 335, 336, 337, 338, 339, 340, 341, ...",333.0,550.0,NaN,"(333.0, 550.0)","Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
25,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,E4,Evidence,Systematic uncertainty,-2.034216,"[914, 915, 916, 917, 918, 919, 920, 921, 922, ...",914.0,1060.0,NaN,"(914.0, 1060.0)","Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
26,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,E5,Evidence,Systematic uncertainty,-1.356144,"[333, 334, 335, 336, 337, 338, 339, 340, 341, ...",333.0,550.0,NaN,"(333.0, 550.0)","Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
27,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,E6,Evidence,Open to evidence,-0.403677,[],-1.0,-1.0,NaN,"(-1.0, -1.0)","Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
28,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,E7,Evidence,Open to evidence,-0.292481,[],-1.0,-1.0,NaN,"(-1.0, -1.0)","Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
29,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,E8,Evidence,Open to evidence,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)","Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
30,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,E9,Evidence,Open to evidence,-2.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)","Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00


In [26]:
full_data['Points'].isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
113    False
114    False
115    False
116    False
117    False
Name: Points, Length: 118, dtype: bool

In [27]:
sum(full_data['Points'].isnull())

0

In [16]:
pd.pivot_table(full_data, index='Article ID', columns='Credibility Indicator Category', values='Points', aggfunc=sum)

Credibility Indicator Category,Evidence,Holistic,Language,Probability,Reasoning
Article ID,,,,,
3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4ec66b45681c86c9301,-1.000000,6.0,-1.000000,-1.075248,-19.203797
47990959103662e94e796d979018922afddc880fb4b867c7ca2ac8aa2146e7c4,NaN,0.0,-2.162065,-5.740532,-27.735788
7360da3cdcf83a48e365821654ef0750810f3483efb8e25ad792eed6f4bf0b6f,-3.525312,-16.0,-2.542770,-2.812520,-17.197880
7d1747829a65cd4dad0faf30a704875a125e9decce3b3bcc82f151193923aad8,-10.568868,1.0,NaN,NaN,NaN
be0b18a87d4370fa579180ef26dcb7080598f27f9ec76181f2cfd851f320da06,-1.900922,4.0,NaN,1.090631,NaN


In [34]:
def five_vectors(df, column_name):
    assert sum(full_data['Points'].isnull()) == 0
    return pd.pivot_table(df, index=column_name, columns='Credibility Indicator Category', values='Points', aggfunc=lambda x: round(sum(x))).fillna(0)

In [35]:
five_vectors(full_data, 'Article ID')

Credibility Indicator Category,Evidence,Holistic,Language,Probability,Reasoning
Article ID,,,,,
3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4ec66b45681c86c9301,-1.0,6.0,-1.0,-1.0,-19.0
47990959103662e94e796d979018922afddc880fb4b867c7ca2ac8aa2146e7c4,0.0,0.0,-2.0,-6.0,-28.0
7360da3cdcf83a48e365821654ef0750810f3483efb8e25ad792eed6f4bf0b6f,-4.0,-16.0,-3.0,-3.0,-17.0
7d1747829a65cd4dad0faf30a704875a125e9decce3b3bcc82f151193923aad8,-11.0,1.0,0.0,0.0,0.0
be0b18a87d4370fa579180ef26dcb7080598f27f9ec76181f2cfd851f320da06,-2.0,4.0,0.0,1.0,0.0


In [36]:
five_vectors(full_data, "Author")

Credibility Indicator Category,Evidence,Holistic,Language,Probability,Reasoning
Author,,,,,
Kerry Grens,-2.0,4.0,0.0,1.0,0.0
Lyubov Stepushova,0.0,0.0,-2.0,-6.0,-28.0
PressTV,-4.0,-16.0,-3.0,-3.0,-17.0
Straits Times,-1.0,6.0,-1.0,-1.0,-19.0
Thomas Williams,-11.0,1.0,0.0,0.0,0.0


## Convert the combined dataframe to CSV and JSON Files

In [37]:
def convert_to_csv(df, column_name):
    combined_df = five_vectors(df, column_name)
    file_name = "combined_{0}.csv".format(column_name)
    combined_df.to_csv(file_name)

In [38]:
def convert_to_json(df, column_name):
    combined_df = five_vectors(df, column_name)
    file_name = "combined_{0}.json".format(column_name)
    combined_df.to_json(file_name)

In [39]:
"combined_{0}.json".format("Author")

'combined_Author.json'